In [2]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *

In [13]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [14]:
version = 0

In [15]:
goal_problems = [sudoku,seeknumbers,minotaur,creek]

In [5]:
for gp in goal_problems:
    query = dict(gp)
    example_1 = nqueens
    example_2 = furniture
    story = query["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    answer = pipeline(example_1,example_2,query)
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    with open(name_of_file,"w") as file:
        file.write(answer)

In [4]:
def asp_try(asp_file, instance):
    solution = ""
    models = None
    success = 0
    try:
        ctl = Control(arguments=["--opt-mode=opt"])
        ctl.load(asp_file)
        ctl.load(instance)
        ctl.add("base", [], solution)
        parts = [("base", [])]
        ctl.ground(parts)      
        try:       
            with ctl.solve(yield_=True, async_=True) as handle: 
                handle.wait(10)
                handle.cancel()
                models = list(iter(handle))
                if len(models) > 0:
                    success = 1
                    #for m in models:
                    #    print(m.symbols(True,True,True))
        except:
            print("Unsatisfiable")
    except:
        print("Parsing or Safety Issues")
    return success, models

In [7]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp")
                }

In [8]:
for gp in goal_problems:
    query = dict(gp)
    example_1 = nqueens
    example_2 = furniture
    story = query["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    print("="*40 + name_of_file + "="*40)
    with open(name_of_file,"r") as f:
        print("Length of File:" + str(len(f.readlines())))
    instance = instances_dict[story]
    success, models = asp_try(name_of_file,instance)
    print("Success: " + str(success))
    if models:
        for m in models:
            print(m.symbols(True,True,True))

========================================generated_solutions\sudoku_from_NQueens_and_furniture_v0.lp========================================
Length of File:12
Success: 0
========================================generated_solutions\seeknumbers_from_NQueens_and_furniture_v0.lp========================================
Length of File:56
Parsing or Safety Issues
Success: 0
========================================generated_solutions\minotaur_from_NQueens_and_furniture_v0.lp========================================
Length of File:25
Success: 0
========================================generated_solutions\creek_from_NQueens_and_furniture_v0.lp========================================
Length of File:20
Parsing or Safety Issues
Success: 0


## End of Pipeline

In [5]:
name_of_file = os.path.join("projects","sudoku","canonical_sudoku.lp")
instance = os.path.join("projects","sudoku","instances","ex01.lp")
success, models = asp_try(name_of_file,instance)

In [6]:
solution = ""
ctl = Control(arguments=["--opt-mode=opt"])
ctl.load(name_of_file)
ctl.load(instance)
ctl.add("base", [], solution)
parts = [("base", [])]
ctl.ground(parts)

In [8]:
with ctl.solve(yield_=True, async_=True) as handle: 
    handle.wait(10)
    handle.cancel()
    models = list(iter(handle))
    if len(models) > 0:
        success = 1

In [7]:
hello = ctl.symbolic_atoms.by_signature('sudoku',3)
count = 0
for h in hello:
    count += 1
    #print(h.symbol)
print("Length: " + str(count))

Length: 241


In [4]:
print(success)

0


In [1]:
for m in models:
    print(m.symbols(True,True,True))

NameError: name 'models' is not defined

In [12]:
constraints_seeknumbers = get_definition_rules(olympics,furniture,copy_seeknumbers)

In [13]:
print(constraints_seeknumbers)

% No option in any category will ever be used more than once.
{C1=C2; C1=C3; C1=C4; C2=C3; C2=C4; C3=C4}=0 :- cell(C1), cell(C2), cell(C3), cell(C4), (C1,C2,C3,C4)!=(C2,C3,C4,C1).
{F1=F2; F1=F3; F1=F4; F2=F3; F2=F4; F3=F4}=0 :- first(F1), first(F2), first(F3), first(F4), (F1,F2,F3,F4)!=(F2,F3,F4,F1).
{Fi1=Fi2; Fi1=Fi3; Fi1=Fi4; Fi2=Fi3; Fi2=Fi4; Fi3=Fi4}=0 :- final(Fi1), final(Fi2), final(Fi3), final(Fi4), (Fi1,Fi2,Fi3,Fi4)!=(Fi2,Fi3,Fi4,Fi1).
{H1=H2; H1=H3; H1=H4; H2=H3; H2=H4; H3=H4}=0 :- hint(H1), hint(H2), hint(H3), hint(H4), (H1,H2,H3,H4)!=(H2,H3,H4,H1).

% The first cell of the path is (1,1).
first(1,1).

% The final cell of the path is (5,5).
final(5,5).

% The cell (X,Y) contains the hint number N.
hint(1,2,3).
hint(2,3,2).
hint(3,4,1).
hint(4,5,4).

% The path must start at the first cell.
path(X,Y) :- first(X,Y).

% The path must end at the final cell.
path(X,Y) :- final(X,Y).

% The path must go through the cells containing the hint numbers.
path(X,Y) :- hint(X,Y,_).

% The 